### Fine-tuning GPT using Compatible Pairs

This file fine-tunes GPT-2 using computed pairs from the matching algorithm that exceed a certain cosine similiarity threshold.

In [ ]:
! pip install -U accelerate
! pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 28.2 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.38.1
    Uninstalling transformers-4.38.1:
      Successfully uninstalled transformers-4.38.1


In [ ]:
import pandas as pd
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import ast

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Load in dataframes for the process

matches_df = pd.read_csv('drive/MyDrive/okcupid_matches.csv')
matches_df = matches_df.rename(columns={'Unnamed: 0': 'Person_id'})

bios_text = pd.read_csv('drive/MyDrive/okcupid_profiles.csv')
bios_text.reset_index(inplace=True)
bios_text = bios_text.rename(columns={'index': 'Person_id'})

## Joining the bios together

bios_text.fillna(' ', inplace=True)
bios_text['allessays'] = bios_text[['essay0', 'essay1', 'essay2', 'essay3', 'essay4', 'essay5', 'essay6', 'essay7', 'essay8']].apply(lambda x: ' '.join(x), axis=1)
bios_text = bios_text[['Person_id', 'allessays']]

## Join two df together on person_id

finetune_df = pd.merge(matches_df, bios_text, on='Person_id').sample(1000)

In [ ]:
## Preparing matches for fine tuning

input_output_pairs = []

for index, row in finetune_df.iterrows():
    if row['matches'] != '[]':
       for match in ast.literal_eval(row['matches']):
           match_id = match[0]
           matching_row = bios_text[bios_text['Person_id'] == match_id]
           ## Add indicator to model what the input and output is
           input_output_pairs.append(F"Input: {row['allessays']}\n")
           input_output_pairs.append(F"Output: {matching_row['allessays'].iloc[0]}\n")

input_output_pairs = "".join(input_output_pairs)

with open("input_output_pairs.txt", "w") as f:
    f.write(input_output_pairs)

In [ ]:
def fine_tune_gpt2(train_file, output_dir):
    '''
    This function fine-tunes Distil-GPT2 model using the created train file.
    
    Inputs: Train file (.txt), output directory (file path)
    Outputs: Model (folder with model components)
    '''
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = GPT2LMHeadModel.from_pretrained("distilgpt2").to(device)
    tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")

    train_dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=train_file,
        block_size=128)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False)

    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,
        num_train_epochs=2,
        per_device_train_batch_size=4,
        save_steps=10_000,
        save_total_limit=2
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

    trainer.train()

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)

In [ ]:
fine_tune_gpt2("input_output_pairs.txt", "fine_tuned_model")

Step,Training Loss
500,4.036500
1000,3.785900
1500,3.637700
2000,3.555200
2500,3.472200
3000,3.418900
3500,3.338800
4000,3.304700
4500,3.253500
5000,3.222900


Checkpoint destination directory fine_tuned_model/checkpoint-10000 already exists and is non-empty. Saving will proceed but saved results may be invalid.


In [ ]:
from google.colab import files
files.download("fine_tuned_model")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
## Saving model to Google Drive

!zip -r /content/fine_tuned_model.zip /content/fine_tuned_model

  adding: content/fine_tuned_model/ (stored 0%)
  adding: content/fine_tuned_model/generation_config.json (deflated 24%)
  adding: content/fine_tuned_model/runs/ (stored 0%)
  adding: content/fine_tuned_model/runs/Mar04_02-22-47_32e0eadc79ed/ (stored 0%)
  adding: content/fine_tuned_model/runs/Mar04_02-22-47_32e0eadc79ed/events.out.tfevents.1709518968.32e0eadc79ed.5092.1 (deflated 63%)
  adding: content/fine_tuned_model/runs/Mar04_02-31-20_32e0eadc79ed/ (stored 0%)
  adding: content/fine_tuned_model/runs/Mar04_02-31-20_32e0eadc79ed/events.out.tfevents.1709519482.32e0eadc79ed.5092.2 (deflated 60%)
  adding: content/fine_tuned_model/runs/Mar04_02-21-43_32e0eadc79ed/ (stored 0%)
  adding: content/fine_tuned_model/runs/Mar04_02-21-43_32e0eadc79ed/events.out.tfevents.1709518904.32e0eadc79ed.5092.0 (deflated 61%)
  adding: content/fine_tuned_model/runs/Mar04_02-32-14_32e0eadc79ed/ (stored 0%)
  adding: content/fine_tuned_model/runs/Mar04_02-32-14_32e0eadc79ed/events.out.tfevents.1709519536.3

In [ ]:
import shutil

In [ ]:
shutil.move("fine_tuned_model.zip", "drive/MyDrive/fine_tuned_model.zip")

'drive/MyDrive/fine_tuned_model.zip'